In [4]:
import numpy as np
import pandas as pd
import glob
import os

In [2]:
paths = sorted(glob.glob("../0.data/results/*.csv"))

In [3]:
len(paths)

40

In [10]:
# 같은 피처를 가진 필드 정리
feature_dict = {}

for path in paths:
    df = pd.read_csv(path).drop(['label', 'timestamp', 'TimeUS'], axis=1)
    msg = os.path.basename(path).split("_")[0]
    for col in df.columns:
        if col in feature_dict:
            feature_dict[col].append(msg)
        else:
            feature_dict[col] = [msg]

# 유일한 feature 목록은 삭제제
for k in list(feature_dict.keys()):
    if len(feature_dict[k]) == 1:
        del feature_dict[k]


In [11]:
inform_df = pd.read_csv("./log_inform_summary.csv")
inform_df.head()

,msg_field,feature,rows,cols,missing,min,max,mean,var,non_zero_rate,unique_count
0,AHR2,Roll,125971,11,False,-4.61000,3.070000,0.246620,9.323946e-01,0.995213,627
1,AHR2,Pitch,125971,11,False,-4.44000,6.960000,0.390088,6.132344e-01,0.990045,776
2,AHR2,Yaw,125971,11,False,71.43000,255.060000,117.313446,4.220584e+03,1.000000,4025
3,AHR2,Alt,125971,11,False,47.48000,80.299995,65.645018,6.839459e+01,1.000000,2962
4,AHR2,Lat,125971,11,False,35.92819,35.928328,35.928282,8.570000e-10,1.000000,1040


In [ ]:
for f, msgs in feature_dict.items():

    for msg in msgs:
        inform_df.loc[inform_df["msg_field"] == msg & inform_df["feature"] == f]

Roll


In [13]:
import pandas as pd

same_feature_rows = []

for feature, msgs in feature_dict.items():
    for msg in msgs:
        row = inform_df[(inform_df["msg_field"] == msg) & (inform_df["feature"] == feature)]
        if not row.empty:
            same_feature_rows.append(row)

same_feature_df = pd.concat(same_feature_rows, ignore_index=True)

# ✅ 컬럼 순서: feature와 msg_field 위치 바꾸기
columns = same_feature_df.columns.tolist()
if "feature" in columns and "msg_field" in columns:
    columns.remove("feature")
    columns.remove("msg_field")
    new_order = ["feature", "msg_field"] + columns
    same_feature_df = same_feature_df[new_order]

# 저장
same_feature_df.to_csv("same_feature_summary.csv", index=False)


In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import os

data_dir = '../0.data/results'  # CSV 파일 경로
output_dir = './results/feature_plots/'  # 그래프 저장 폴더
os.makedirs(output_dir, exist_ok=True)

for feature, msgs in feature_dict.items():
    plt.figure(figsize=(12, 6))

    for msg in msgs:
        path = os.path.join(data_dir, f"{msg}_merged.csv")
        try:
            df = pd.read_csv(path)
            if 'timestamp' in df.columns and feature in df.columns:
                plt.plot(df['timestamp'], df[feature], label=msg)
        except Exception as e:
            print(f"{msg} 파일 처리 중 오류: {e}")

    plt.title(f"{feature} - 단순 비교 그래프")
    plt.xlabel('Timestamp')
    plt.ylabel(feature)
    plt.legend()
    plt.savefig(os.path.join(output_dir, f"{feature}_comparison.png"))
    plt.close()
    break

        

/tmp/ipykernel_3131750/2899557256.py:25: UserWarning: Glyph 45800 (\N{HANGUL SYLLABLE DAN}) missing from current font.
  plt.savefig(os.path.join(output_dir, f"{feature}_comparison.png"))
/tmp/ipykernel_3131750/2899557256.py:25: UserWarning: Glyph 49692 (\N{HANGUL SYLLABLE SUN}) missing from current font.
  plt.savefig(os.path.join(output_dir, f"{feature}_comparison.png"))
/tmp/ipykernel_3131750/2899557256.py:25: UserWarning: Glyph 48708 (\N{HANGUL SYLLABLE BI}) missing from current font.
  plt.savefig(os.path.join(output_dir, f"{feature}_comparison.png"))
/tmp/ipykernel_3131750/2899557256.py:25: UserWarning: Glyph 44368 (\N{HANGUL SYLLABLE GYO}) missing from current font.
  plt.savefig(os.path.join(output_dir, f"{feature}_comparison.png"))
/tmp/ipykernel_3131750/2899557256.py:25: UserWarning: Glyph 44536 (\N{HANGUL SYLLABLE GEU}) missing from current font.
  plt.savefig(os.path.join(output_dir, f"{feature}_comparison.png"))
/tmp/ipykernel_3131750/2899557256.py:25: UserWarning: Glyph 4